# IRAM Debug - Check What's Actually Loaded
Run this to see what instructions are actually in IRAM at each address

In [ ]:
# Expected instructions from program
PROGRAM = [
    0x00000093,  #  0: addi x1, x0, 0
    0x01F00113,  #  1: addi x2, x0, 31
    0x00400513,  #  2: addi x10, x0, 4
    0x00000193,  #  3: addi x3, x0, 0
    0x0021A4B3,  #  4: slt  x9, x3, x2
    0x04048263,  #  5: beq  x9, x0, +68
    0x40310233,  #  6: sub  x4, x2, x3
    0x00000293,  #  7: addi x5, x0, 0
    0x0042A4B3,  #  8: slt  x9, x5, x4
    0x02048663,  #  9: beq  x9, x0, +44
    0x00229313,  # 10: slli x6, x5, 2
    0x00608333,  # 11: add  x6, x1, x6
    0x00032383,  # 12: lw   x7, 0(x6)
    0x00432403,  # 13: lw   x8, 4(x6)
    0x007424B3,  # 14: slt  x9, x8, x7
    0x00048663,  # 15: beq  x9, x0, +12
    0x00832023,  # 16: sw   x8, 0(x6)
    0x00732223,  # 17: sw   x7, 4(x6)
    0x00128293,  # 18: addi x5, x5, 1
    0xFC000AE3,  # 19: beq  x0, x0, -44
    0x00118193,  # 20: addi x3, x3, 1
    0xFA000EE3,  # 21: beq  x0, x0, -68
    0xDEADC637,  # 22: lui  x12, 0xDEADC
    0xEAF60613,  # 23: addi x12, x12, -337
    0x10000693,  # 24: addi x13, x0, 256
    0x00C6A023,  # 25: sw   x12, 0(x13)
    0x00000063,  # 26: beq  x0, x0, 0
]

print("Reading IRAM contents...\n")
print("Expected vs Actual:")
print("Idx | ByteAddr | Expected   | Actual     | Match")
print("-" * 55)

mismatches = []
for i in range(30):  # Check first 30 words
    byte_addr = i * 4
    actual = iram.read(byte_addr)
    expected = PROGRAM[i] if i < len(PROGRAM) else 0x00000013  # NOP if beyond program
    match = "✓" if actual == expected else "✗ MISMATCH"
    print(f" {i:2d} |   0x{byte_addr:02X}   | 0x{expected:08X} | 0x{actual:08X} | {match}")
    
    if actual != expected:
        mismatches.append((i, byte_addr, expected, actual))
        # Check if this actual value matches some other expected instruction
        for j, prog_instr in enumerate(PROGRAM):
            if actual == prog_instr:
                print(f"      └── Actual value matches instruction {j} (should be at 0x{j*4:02X})")
                break

print("\n" + "=" * 55)
if not mismatches:
    print("✓ All instructions loaded correctly!")
else:
    print(f"✗ Found {len(mismatches)} mismatched addresses")
    print("\nPattern analysis:")
    if len(mismatches) >= 2:
        shift = None
        for i, byte_addr, expected, actual in mismatches:
            for j, prog_instr in enumerate(PROGRAM):
                if actual == prog_instr:
                    offset = j - i
                    print(f"  Address 0x{byte_addr:02X}: has instruction from index {j} (offset: {offset:+d} words, {offset*4:+d} bytes)")
                    if shift is None:
                        shift = offset
                    break
        if shift is not None:
            print(f"\n  → Instructions appear shifted by {shift:+d} words ({shift*4:+d} bytes)")
print("=" * 55)

In [ ]:
# Find where specific instructions actually are
print("\nSearching for key instructions:")
print("Instruction 21 (0xFA000EE3 - beq back to outer loop):")
print("Instruction 22 (0xDEADC637 - done marker):")
print()

for addr in range(0, 120, 4):  # Search first 30 words
    val = iram.read(addr)
    if val == 0xFA000EE3:
        print(f"  Found 0xFA000EE3 (inst 21) at byte address 0x{addr:02X} ({addr}) - SHOULD be at 0x{21*4:02X} ({21*4})")
    if val == 0xDEADC637:
        print(f"  Found 0xDEADC637 (inst 22) at byte address 0x{addr:02X} ({addr}) - SHOULD be at 0x{22*4:02X} ({22*4})")